### create training set of trigrams (x1, x2, y)

In [1]:
import torch

In [12]:
words = open("names.txt", "r").read().splitlines()
print(len(words))

# a to z
chars = sorted(set(''.join(words)))
s_to_i = {s: i + 1 for i, s in enumerate(chars)}
s_to_i["."] = 0
i_to_s = {i: s for s, i in s_to_i.items()}

32033


In [3]:
x1s, x2s, ys = [], [], []

for word in words:
	chs = ['.'] + list(word) + ['.']
	for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
		index_1 = s_to_i[ch1]
		index_2 = s_to_i[ch2]
		index_3 = s_to_i[ch3]
		x1s.append(index_1)
		x2s.append(index_2)
		ys.append(index_3)

x1s = torch.tensor(x1s)
x2s = torch.tensor(x2s)
ys = torch.tensor(ys)

In [4]:
N = torch.zeros((27, 27, 27), dtype=torch.int32)

In [5]:
for word in words:
	chs = ['.'] + list(word) + ['.']
	for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
		index_1 = s_to_i[ch1]
		index_2 = s_to_i[ch2]
		index_3 = s_to_i[ch3]

		N[index_1, index_2, index_3] += 1


In [6]:
print(f"N shape: {N.shape}")
print(f"N dtype: {N.dtype}")
print(f"Total count: {N.sum().item()}")
print(f"Non-zero entries: {(N > 0).sum().item()}")
print(f"Max value: {N.max().item()}")
print(f"Sample of N[0, :, :] (first character is '.'):")
print(N[0, :5, :5])  # Show a small slice

N shape: torch.Size([27, 27, 27])
N dtype: torch.int32
Total count: 196113
Non-zero entries: 6037
Max value: 1714
Sample of N[0, :, :] (first character is '.'):
tensor([[  0,   0,   0,   0,   0],
        [  0, 207, 190,  31, 366],
        [  0, 169,   0,   0,   0],
        [  0, 628,   0,   0,   0],
        [  0, 700,   0,   0,   0]], dtype=torch.int32)


In [7]:
import plotly.graph_objects as go
import numpy as np

# Get non-zero entries - CORRECTED
nonzero_indices = torch.nonzero(N > 0)  # Returns Nx3 tensor
i_indices = nonzero_indices[:, 0]
j_indices = nonzero_indices[:, 1]
k_indices = nonzero_indices[:, 2]
values = N[i_indices, j_indices, k_indices].float()

# Create hover text with actual characters
hover_text = [f"'{i_to_s[i.item()]}{i_to_s[j.item()]}{i_to_s[k.item()]}': {v.item()}"
              for i, j, k, v in zip(i_indices, j_indices, k_indices, values)]

fig = go.Figure(data=[go.Scatter3d(
    x=i_indices.numpy(),
    y=j_indices.numpy(),
    z=k_indices.numpy(),
    mode='markers',
    marker=dict(
        size=np.log(values.numpy() + 1) * 2,
        color=np.log(values.numpy() + 1),
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Log Frequency"),
        opacity=0.8
    ),
    text=hover_text,
    hoverinfo='text'
)])

fig.update_layout(
    title='Interactive 3D Trigram Visualization',
    scene=dict(
        xaxis_title='First Character',
        yaxis_title='Second Character',
        zaxis_title='Third Character',
    ),
    width=900,
    height=700
)

fig.show()

### Create dataset

In [ ]:
# create the dataset
x1s, x2s, ys = [], [], []
for w in words:
	chs = ['.'] + list(w) + ['.']
	for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
		index_1 = s_to_i[ch1]
		index_2 = s_to_i[ch2]
		index_3 = s_to_i[ch3]
		x1s.append(index_1)
		x2s.append(index_2)
		ys.append(index_3)
x1s = torch.tensor(x1s)
x2s = torch.tensor(x2s)
ys = torch.tensor(ys)
num = ys.nelement()
print('number of examples: ', num)

number of examples:  196113


### Grad desc

In [22]:
import torch.nn.functional as F
x1_enc = F.one_hot(x1s, num_classes = 27).float()
x1_enc.size()

torch.Size([196113, 27])

### Different approaches for a trigram NN
**Approach 1**
- Use 3D indexing
- Direct translation to the count-based approach (although that isn't there in this notebook)

**Approach 2**
- Concatenate inputs; use a `54x27` matrix instead of a `27x27x27` matrix, effectively concatenating the two inputs x1 and x2
  
**Why there is no one-hot encoding?**

Let's say we have 'e' (index 5) and 'm' (index 13):

```python
W = torch.randn((27, 27, 27))  # Trigram weight tensor

# What we want: the row corresponding to (5, 13)
# This should give us the 27 probabilities for the next character
```

If you wanted to do this with one-hot encoding, you'd need:
```python
# One-hot encode both characters
x1_enc = F.one_hot(torch.tensor([5]), num_classes=27).float()   # [0,0,0,0,0,1,0,...]
x2_enc = F.one_hot(torch.tensor([13]), num_classes=27).float()  # [0,0,0,...,1,0,...]

# Now you need to do a "double selection":
# This would require Einstein summation or einsum:
logits = torch.einsum('i,j,ijk->k', x1_enc[0], x2_enc[0], W)
# What this does:
# Sum over i and j where x1_enc[i]=1 and x2_enc[j]=1
# = 0*0*W[0,0,:] + 0*0*W[0,1,:] + ... + 1*1*W[5,13,:] + ...
# = W[5, 13, :]  ← It selects the (5,13) "row"!
```

But that's complicated! Direct indexing does the same thing:
```python
# Method 1: One-hot + einsum (complex)
x1_enc = F.one_hot(torch.tensor([5]), num_classes=27).float()
x2_enc = F.one_hot(torch.tensor([13]), num_classes=27).float()logits = torch.einsum('i,j,ijk->k', x1_enc[0], x2_enc[0], W)

# Method 2: Direct indexing (simple)
logits = W[5, 13]  # Exactly the same result!
```

In [ ]:
# # initialize the 'network'
# g = torch.Generator().manual_seed(2147483647)
# W = torch.randn((54, 27), generator=g, requires_grad=True)

# for k in range(200):
# 	# forward pass
# 	# Use x1s and x2s directly as indices into W
# 	x1enc = F.one_hot(x1s, num_classes=27).float() # input to the network: one-hot encoding
# 	x2enc = F.one_hot(x2s, num_classes=27).float()
# 	xenc = torch.cat([x1enc, x2enc], dim=1)
# 	logits = xenc @ W # predict log-counts
# 	counts = logits.exp() # counts, equivalent to N
# 	probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
# 	loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
# 	print(loss.item())

# 	# backward pass
# 	W.grad = None # set to zero the gradient
# 	loss.backward()

# 	# update
# 	W.data += -50 * W.grad


# Initialize W as a 3D tensor (27 x 27 x 27)
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27, 27), generator=g, requires_grad=True)

for k in range(200):
    # forward pass
    # Use x1s and x2s directly as indices into W
    logits = W[x1s, x2s]  # Shape: (num, 27) - for each (x1, x2) pair, get 27 logits
    counts = logits.exp()  # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True)  # probabilities for next character
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    print(loss.item())

    # backward pass
    W.grad = None
    loss.backward()

    # update
    W.data += -50 * W.grad

3.7331693172454834
3.657506227493286
3.586538791656494
3.5202581882476807
3.45867919921875
3.4017715454101562
3.3494110107421875
3.3013699054718018
3.2573297023773193
3.2169063091278076
3.179690361022949
3.1452884674072266
3.113347291946411
3.083566427230835
3.0556976795196533
3.0295352935791016
3.004910945892334
2.981682777404785
2.959730863571167
2.9389517307281494
2.919255018234253
2.9005610942840576
2.882798433303833
2.8659026622772217
2.849816083908081
2.8344857692718506
2.819861888885498
2.8058998584747314
2.79255747795105
2.779794692993164
2.767575263977051
2.7558653354644775
2.744631290435791
2.733844041824341
2.7234747409820557
2.7134971618652344
2.703887462615967
2.694622755050659
2.6856820583343506
2.6770458221435547
2.6686973571777344
2.6606192588806152
2.6527960300445557
2.6452150344848633
2.63786244392395
2.6307260990142822
2.623795747756958
2.617060899734497
2.6105117797851562
2.604140043258667
2.5979371070861816
2.591895580291748
2.5860085487365723
2.5802688598632812
2.

### Sample from the NN

In [31]:
# g = torch.Generator().manual_seed(2147483647)

# for i in range(5):
# 	out = []
# 	index = 0
# 	while True:
# 		x_enc = F.one_hot(torch.tensor([index]), num_classes=27).float()
# 		logits = x_enc @ W
# 		counts = logits.exp()
# 		p = counts / counts.sum(1, keepdims=True)

# 		index = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
# 		out.append(i_to_s[index])
# 		if index == 0:
# 			break
# 	print(''.join(out))

g = torch.Generator().manual_seed(2147483647)

for i in range(5):
	out = []
	index1 = 0  # First character (start token '.')
	index2 = 0  # Second character (also start token '.')

	while True:
		# Use both previous characters to index into W
		logits = W[index1, index2]  # Shape: (27,) - gives probabilities for next char
		counts = logits.exp()
		p = counts / counts.sum()  # No need for keepdims since it's already 1D
		# Sample the next character
		index3 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
		out.append(i_to_s[index3])

		if index3 == 0:  # Stop token
			break

		# Slide the window: the new becomes old
		index1 = index2
		index2 = index3

	print(''.join(out))

zexzdfzjglkuriana.
otxhkmvlzimjtna.
orakayk.
ka.
oa.
